This notebook walks through how to make graphs in this repo. You can copy this example.ipynb and rename it for your own graphing work. <b> Please do each graph in a seperate notebook!</b>

1. Import the appropriate packages. We'll use plotly.graph_objects for making all of our graphs.

In [1]:
import datetime as dt
import plotly.graph_objects as go
import plotly.io as pio
import pandas as pd
import utils.utils as ut
import utils.graph_templates

from fredapi import Fred

2. We'll get data using the fredapi package. Set the path to the text file with your API key

In [2]:
# Call the graph the exact same thing as its notebook (minus the ipynb suffix) NAME FOR THE FILE, not the title
GRAPH_NAME = "jobs_added"

API_KEY_PATH = ut.get_repo_root() / "fred_api_key.txt" 

fred = Fred(api_key_file = API_KEY_PATH)

3. Set the fed_2025 template as default

In [3]:
pio.templates.default = 'fed_2025'

# Now is a good time to set the path to the graph output folder!
GRAPH_OUTPUT_PATH = ut.get_repo_root() / "figures"

4. Use the fredapi to get the data and prepare it for graphing. Documentation on the optional parameters that can be passed to the get_series called are found here (the documentation in fredapi is out of date). 

https://fred.stlouisfed.org/docs/api/fred/series_observations.html#Description

If you get data from somewhere else thats fine too! Put the raw csv in the "raw_data" folder and read it in here. Make sure not to edit the raw data, just transform and graph it.

In [4]:
# It's good practice to store the series codes in a dictionary with their names
series_codes = {
    "Labor Force Participation Rate": "CIVPART",
    "Employment-Population Ratio": "EMRATIO",
}

In [22]:
today = dt.date.today()

non_farm_srs = fred.get_series(
    series_id=series_codes["Employment-Population Ratio"],
    observation_start=dt.date(2024, 1, 1),
    observation_end=today,
    frequency='m',
).rename("Employment-Population Ratio")


labor_srs = fred.get_series(
    series_id=series_codes["Labor Force Participation Rate"],
    observation_start=dt.date(2024, 1, 1),
    observation_end=today,
    frequency='m',
).rename("Labor Force Participation Rate")


joined_df = pd.concat(
    [non_farm_srs, labor_srs],
    axis=1,
    join='outer'
)

joined_df.index.name = "Date"

joined_df.tail()


,Employment-Population Ratio,Labor Force Participation Rate
Date,,
2025-04-01,60.0,62.6
2025-05-01,59.7,62.4
2025-06-01,59.7,62.3
2025-07-01,59.6,62.2
2025-08-01,59.6,62.3


5. Now that all our data is ready, make the graph and have it save itself as a .html file to graph_output whenver the notebooks is rerun. The name of the file should exactly match the notebook name. For instance, this file "example.ipynb" produces the graph "example.html." Nice work, you made a graph! 

In [27]:
# First make the figure
fig = go.Figure()

for col in joined_df.columns:
    fig.add_trace(
        go.Scatter(
            x=joined_df.index,
            y=joined_df[col],
            mode='lines',
            name=col
        )
    )

ut.add_end_labels(fig = fig)



# Update the titles, using the html tage <sup> for a subtitle 
fig.update_layout(
    title = dict(text = "LFPR & EPOP<br><sup>Monthly, Seasonally Adjusted </sup>"),
    xaxis_title="Date",
    yaxis_title="",
    legend=dict(orientation="v"),
    width=800,
    height=500
)



# This is graph specific, but here we want the y-axis to be percent signs 
fig.update_yaxes(
    tickformat=".1f",
    ticksuffix="%"
)

# Again, graph specific, we have a mutliyear series and want tick marks to be years
fig.update_xaxes(
    type='date',
    dtick="M4",
)


ut.add_end_labels(fig = fig)

# Show our figure (Dimensions may be off on different screen sizes)
fig.show()

# This should be the same for EVERY GRAPH!
# Save it to the graph_output folder with the name matching the file, as HTML
fig.write_html(GRAPH_OUTPUT_PATH / f"{GRAPH_NAME}.html")